# Simulaciones

In [ ]:
import numpy as np
import time
import multiprocessing as mp
from joblib import Parallel, delayed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def square(x):
    return x ** 2

def process_chunk(chunk):
    return [square(x) for x in chunk]

def parallel_square(arr, num_processes):
    chunk_size = len(arr) // num_processes
    chunks = [arr[i:i + chunk_size] for i in range(0, len(arr), chunk_size)]
    with mp.Pool(processes=num_processes) as pool:
        results = pool.map(process_chunk, chunks)
    return np.concatenate(results)

def run_experiment(arr):
    times = {}
    
    # For loop tradicional
    start_time = time.time()
    result = [square(x) for x in arr]
    times['For loop'] = time.time() - start_time

    # Vectorización
    start_time = time.time()
    result = arr ** 2
    times['Vectorización'] = time.time() - start_time

    # Joblib
    start_time = time.time()
    result = Parallel(n_jobs=-1)(delayed(square)(x) for x in arr)
    times['Joblib'] = time.time() - start_time

    # Multiprocesamiento
    num_processes = mp.cpu_count()
    start_time = time.time()
    result_parallel = parallel_square(arr, num_processes)
    times['Multiprocesamiento'] = time.time() - start_time

    return times

def run_simulations(n_simulations=10, array_size=1000000):
    results = []
    for _ in range(n_simulations):
        arr = np.random.rand(array_size)
        times = run_experiment(arr)
        results.append(times)
    return pd.DataFrame(results)

def analyze_results(df):
    stats = df.agg(['mean', 'std', 'min', 'max'])
    print("Estadísticas básicas:")
    print(stats)
    return stats

def plot_results(df, stats):
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df.melt(), x='variable', y='value')
    plt.title('Comparación de tiempos de ejecución')
    plt.xlabel('Método')
    plt.ylabel('Tiempo (segundos)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 6))
    stats.T['mean'].plot(kind='bar', yerr=stats.T['std'], capsize=5)
    plt.title('Tiempo medio de ejecución con desviación estándar')
    plt.xlabel('Método')
    plt.ylabel('Tiempo (segundos)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    n_simulations = 10
    array_size = 1000000

    print(f"Ejecutando {n_simulations} simulaciones con arreglos de tamaño {array_size}...")
    results_df = run_simulations(n_simulations, array_size)
    
    print("\nGuardando resultados en CSV...")
    results_df.to_csv('simulation_results.csv', index=False)
    
    print("\nAnalizando resultados...")
    stats = analyze_results(results_df)
    
    print("\nGenerando gráficos...")
    plot_results(results_df, stats)
    
    print("\n¡Experimento completado!")
    print("Los resultados se han guardado en 'simulation_results.csv'")
    print("Los gráficos se han mostrado en pantalla.")